In [1]:
%reload_ext autoreload
%autoreload 2

import socket
import psutil
import sys 
import os
from typing import Any
from functools import partial
import json
from pprint import pprint
from collections import OrderedDict
from datetime import datetime
import os
import glob

# lib_path = ['/fsx_0/user/tranx/experiments']
import_paths = [
    # "/fsx_0/user/tranx/experiments",
    # "/fsx_0/user/tranx",
    # "/fsx_0/user/tranx/rsync",
    "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws",
    # "/fsx_0/user/tranx/experiments/lib"
]
for path in import_paths:
    if path not in sys.path:
        sys.path.append(path)
        
from launch_evals import read_eval_results

import pandas as pd
pd.set_option('display.max_rows', 100)

def get_trainer_state(output_dir: str):
    checkpoint_dirs = glob.glob(f"{output_dir}/checkpoint-*")
    checkpoints = [int(d.split("/")[-1].split("-")[-1]) for d in checkpoint_dirs]
    if len(checkpoints) == 0:
        return None, None 
    
    checkpoint_latest = max(checkpoints)
    # print(f"{checkpoint_latest=}")

    trainer_state_file = os.path.join(
        output_dir,
        f"checkpoint-{checkpoint_latest}",
        "trainer_state.json"
    )

    with open(trainer_state_file, 'r') as file:
        state = json.load(file)
        
    return checkpoint_latest, state

# MM10.2

In [8]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/LLama3.3_70B_ViTG_504px_R1",
    steps=[10000],
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE_adversarial,POPE_popular,POPE_random
10000,53.78,47.21,36.9,69.49,35.31,67.63,59.95,55.45,40.68,71.31,NaN,NaN,NaN


# 8B Midtraining

In [60]:
# base_dir = "/fsx_3/bucket/tranx/checkpoints/vev0"
base_dir = "/fsx_3/bucket/cyprien/checkpoints/data_ablations_mm10"
output_dirs = glob.glob(f"{base_dir}/Llama3.1*")
for d in output_dirs:
    if "select" in d:
        print(d)
        name = d.split("/")[-1]
        # checkpoint_latest, state = get_trainer_state(d)
        
        # print(f"{name}, {checkpoint_latest=}")
        print(f"{name}")

/fsx_3/bucket/cyprien/checkpoints/data_ablations_mm10/Llama3.1_8B_ViTG_392_stage1_base_vision_select2
Llama3.1_8B_ViTG_392_stage1_base_vision_select2
/fsx_3/bucket/cyprien/checkpoints/data_ablations_mm10/Llama3.1_8B_ViTG_392_stage1_base_vision_select3
Llama3.1_8B_ViTG_392_stage1_base_vision_select3


In [ ]:
/fsx_3/bucket/tranx/checkpoints/vev0/Llama3.1_8B_ViTG_392_midtrain_perceiver_17k5_R3_vision_select3

In [88]:
df = read_eval_results(
    watch_path="/fsx_3/bucket/cyprien/checkpoints/data_ablations_mm10/Llama3.1_8B_ViTG_base_VE_stage1_base",
    # watch_path="/fsx_3/bucket/cyprien/checkpoints/data_ablations_mm10/Llama3.1_8B_ViTG_stage1_from_midtrained_perceiver_17k5_base",
    # steps=[3200],
    # watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/MH24_70B_ViTG_392px_R1",
    # watch_path="/fsx_3/bucket/tranx/checkpoints/vev0/Llama3.1_8B_ViTG_392_midtrain_perceiver_17k5_R3_vision_select3",
    # watch_path="/fsx_3/bucket/cyprien/checkpoints/data_ablations_mm10/Llama3.1_8B_ViTG_392_stage1_base_vision_select3",
    read_format="report_mmai",
    # base_eval_output_dir="evals_master"
)

df.insert(0, 'Avg', df.iloc[:, :-3].mean(axis=1))
df

,Avg,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE_adversarial,POPE_popular,POPE_random
200,39.028,41.37,23.91,28.6,40.58,35.12,57.32,35.82,42.50,30.04,55.02,NaN,NaN,NaN
400,47.372,44.36,44.74,31.6,49.35,41.94,63.65,55.15,45.34,35.02,62.57,NaN,NaN,NaN
600,48.262,44.03,48.81,30.5,51.39,42.93,64.70,57.43,44.60,34.94,63.29,NaN,NaN,NaN
800,49.372,45.24,52.77,30.8,54.44,42.07,65.29,59.49,43.55,35.50,64.57,NaN,NaN,NaN
1000,49.495,44.58,54.97,30.4,55.28,42.31,65.65,59.84,43.81,34.58,63.53,NaN,NaN,NaN
1200,50.227,44.58,56.80,31.1,56.12,41.14,65.79,60.88,45.03,36.50,64.33,NaN,NaN,NaN
1400,50.398,44.69,57.81,31.5,55.47,42.40,66.10,61.26,44.18,35.95,64.62,NaN,NaN,NaN
1600,50.716,45.02,58.99,30.9,51.75,42.91,66.06,60.69,46.41,38.84,65.59,NaN,NaN,NaN
1800,50.760,47.01,58.36,30.6,50.74,42.89,66.40,61.81,44.79,39.13,65.87,NaN,NaN,NaN
2000,51.085,44.91,60.16,31.6,54.92,41.85,67.03,62.23,45.37,38.52,64.26,NaN,NaN,NaN


In [96]:
df = read_eval_results(
    # watch_path="/fsx_3/bucket/tranx/checkpoints/vev0/Llama3.1_8B_ViTG_392_midtrain_perceiver_17k5_R3_vision_select3",
    watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/MH24_70B_ViTG_392px_R1",
    # steps=[17500, 20000],
    read_format="report_mmai",
    base_eval_output_dir="evals_master"
)
df.insert(0, 'Avg', df.iloc[:, :-3].mean(axis=1))
df

,Avg,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE_adversarial,POPE_popular,POPE_random
600,54.827,57.44,45.51,34.9,70.76,35.98,67.40,56.96,61.96,45.29,72.07,NaN,NaN,NaN
800,55.678,55.67,48.56,34.9,71.02,37.34,67.48,60.37,61.57,47.50,72.37,NaN,NaN,NaN
1000,56.406,55.11,52.81,34.9,71.96,37.54,68.26,61.15,62.27,48.24,71.82,NaN,NaN,NaN
1200,57.145,56.89,54.87,35.5,72.18,38.72,68.36,61.92,62.20,48.40,72.41,NaN,NaN,NaN
1400,57.035,55.33,55.07,36.3,72.18,37.59,68.32,62.59,62.25,48.66,72.06,NaN,NaN,NaN
1600,57.793,56.33,56.03,35.7,73.74,40.03,68.75,62.74,62.63,48.89,73.09,NaN,NaN,NaN
1800,57.741,58.22,54.87,36.2,72.41,40.59,69.01,62.40,62.55,48.86,72.30,NaN,NaN,NaN
2000,58.046,58.33,57.11,35.3,72.12,40.53,68.75,63.09,62.30,49.20,73.73,NaN,NaN,NaN
2200,57.903,57.33,56.05,37.0,73.41,40.14,68.65,63.12,61.89,48.32,73.12,NaN,NaN,NaN
2400,57.861,55.89,57.52,37.2,72.77,39.42,68.63,63.63,61.67,48.58,73.30,NaN,NaN,NaN


In [11]:
for d in output_dirs:
    print(d)
    df = read_eval_results(
        watch_path=d,
        steps=[4500],
        read_format="report_mmai"
    )
    print(df)

/fsx_3/bucket/tranx/checkpoints/vev0/Llama3.1_8B_BigG-504-f572374596_warmup
       mmmu  docvqa  mathvista  ai2d  chartqa    vqa  textvqa  \
4500  46.02   56.28       30.9  57.9    43.91  67.32    58.66   

      infographics_w_ocr  infographics  mmbench  
4500               48.12         42.29    67.74  
/fsx_3/bucket/tranx/checkpoints/vev0/Llama3.1_8B_BigG-504-0712_warmup
       mmmu  docvqa  mathvista   ai2d  chartqa    vqa  textvqa  \
4500  42.92   56.52       29.9  50.74    41.97  67.94    61.04   

      infographics_w_ocr  infographics  mmbench  
4500               46.45         41.54    66.99  
/fsx_3/bucket/tranx/checkpoints/vev0/Llama3.1_8B_ViTG_midtrain_R2


FileNotFoundError: [Errno 2] No such file or directory: '/fsx_3/bucket/tranx/checkpoints/vev0/Llama3.1_8B_ViTG_midtrain_R2/evals'

# Perceiver Sizing

In [32]:
base_dir = "/fsx_3/bucket/tranx/checkpoints/perceiver_sizing"
output_dirs = glob.glob(f"{base_dir}/Llama3*")
for d in output_dirs:
    print(d)
    name = d.split("/")[-1]
    checkpoint_latest, state = get_trainer_state(d)
    
    print(f"{name}, {checkpoint_latest=}")

/fsx_3/bucket/tranx/checkpoints/perceiver_sizing/Llama3.1_70B_ViTG_layers18_dim4096_heads32_latents64_bz64_step4
Llama3.1_70B_ViTG_layers18_dim4096_heads32_latents64_bz64_step4, checkpoint_latest=10600
/fsx_3/bucket/tranx/checkpoints/perceiver_sizing/Llama3.1_70B_ViTG_layers22_dim4096_heads32_latents64_bz64_step8
Llama3.1_70B_ViTG_layers22_dim4096_heads32_latents64_bz64_step8, checkpoint_latest=11200
/fsx_3/bucket/tranx/checkpoints/perceiver_sizing/Llama3.1_70B_ViTG_layers22_dim4096_heads32_latents128_bz32_step8
Llama3.1_70B_ViTG_layers22_dim4096_heads32_latents128_bz32_step8, checkpoint_latest=10000
/fsx_3/bucket/tranx/checkpoints/perceiver_sizing/Llama3.1_70B_ViTG_layers26_dim4096_heads32_latents64_bz64_step4
Llama3.1_70B_ViTG_layers26_dim4096_heads32_latents64_bz64_step4, checkpoint_latest=10300
/fsx_3/bucket/tranx/checkpoints/perceiver_sizing/Llama3.1_70B_ViTG_layers14_dim4096_heads32_latents64_bz64_step4
Llama3.1_70B_ViTG_layers14_dim4096_heads32_latents64_bz64_step4, checkpoint_l

In [34]:
read_eval_results(
    watch_path="/fsx_3/bucket/tranx/checkpoints/perceiver_sizing/Llama3.1_70B_ViTG_layers22_dim4096_heads32_latents128_bz32_step8",
    steps=[10000],
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE_adversarial,POPE_popular,POPE_random
10000,55.89,56.16,34.6,74.64,47.39,70.26,62.54,62.03,48.83,74.79,NaN,NaN,NaN


# MM10.1 Prod

In [10]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH22final_70B_ViTH_336px_R1_idl",
    steps=[17800],
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
17800,61.56,63.96,39.4,75.45,44.57,71.45,68.74,61.58,51.4,73.33


In [11]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10.1/MM10.1_Stage1_70B/MH22final_70B_ViTH_336px_R1_recap_64nodes_fix",
    steps=[15500],
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
15500,60.33,62.73,39.5,74.45,45.17,71.65,68.39,61.42,50.36,73.93


# MM10.1 - 8B

# MM10.1 - MoE

In [2]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/tranx/moe/70B_stage2_baseline",
    steps=[3700],
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
3700,56.89,70.57,45.4,76.2,46.15,74.98,72.1,60.58,57.04,75.93


In [3]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/tranx/moe/70B_moe_stage2_upcycling_22x8x2",
    steps=[3700],
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
3700,55.22,70.06,45.2,76.62,56.65,74.28,71.41,64.53,59.56,76.14


# MM9.1

In [3]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm9/MM9-Stage2-70B/MH19_336px_128nodes_exp30",
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
8800,59.11,81.17,53.2,80.12,58.77,77.05,75.54,71.65,66.88,77.38


In [4]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm9/MM9-Stage2-70B/MH19_336px_128nodes_exp30_3x3",
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
8800,58.11,74.97,53.8,79.34,57.59,76.71,74.8,65.69,56.44,77.83


# MM9.2

In [5]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm9.2/MM9.2_Stage1_70B/LLama3.1_70B_ViTH_336px_R1_recap",
    steps=[17100],
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
17100,56.56,60.96,35.6,75.62,51.94,70.33,65.23,61.76,50.06,75.04


In [9]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm9.2/MM9.2_Stage2_70B/Llama3.1_70B_ViTH_336px_exp28_plus_v2_20241111",
    # steps=[17100],
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,54.11,76.61,39.9,72.34,53.98,71.47,69.07,63.04,57.96,73.00
200,55.00,77.17,40.4,72.60,53.43,71.86,69.38,63.53,58.76,73.18
300,55.22,77.40,41.5,72.57,53.00,72.12,69.50,63.44,58.53,72.98
400,56.33,77.40,42.0,73.09,53.57,72.25,69.93,63.83,58.54,73.12
500,55.00,78.00,41.9,72.70,52.63,72.04,70.25,63.46,58.73,72.88
600,54.44,77.84,41.5,73.02,53.00,72.39,69.83,63.70,58.84,73.27
700,55.44,78.46,41.6,73.02,52.07,72.50,69.72,63.84,59.01,73.77
800,56.11,78.45,42.8,73.22,52.56,72.75,69.89,63.58,59.32,74.22
900,55.67,78.96,43.4,73.41,52.72,72.66,70.09,63.96,59.51,74.47
1000,54.56,78.74,42.3,73.35,52.38,72.66,69.71,63.92,58.89,74.45


In [2]:
df = read_eval_results(
    # watch_path="/fsx_0/checkpoints/mm9.2/MM9.2_Stage2_70B/Llama3.1_70B_ViTH_336px_exp28_plus_v2_20241111_64nodes",
    watch_path="/fsx_0/checkpoints/mm9.2/MM9.2_Stage2_70B/Llama3.1_70B_ViTH_336px_exp28_plus_v4_20241116_64nodes",
    # steps=[17100],
    read_format="report_mmai"
)

In [5]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm9.2/MM9.2_SFT/MM9p2_19700_prod_recipe_v2",
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
2000,54.89,84.26,44.1,69.69,63.65,55.80,71.97,68.40,64.83,75.35
4000,54.33,84.67,44.4,70.21,59.52,56.98,72.88,68.82,66.03,73.71
6000,55.33,85.00,43.1,71.34,57.63,57.70,74.19,69.24,66.17,74.36
8000,55.56,85.42,44.2,71.76,60.74,56.83,74.42,68.60,66.04,74.05
10000,56.33,85.09,44.3,72.83,60.21,56.12,74.46,68.89,66.02,73.99
12000,55.22,85.48,43.3,72.02,60.37,56.76,74.66,69.16,65.97,73.69
14000,NaN,85.43,43.6,72.47,60.38,NaN,74.78,68.92,65.94,74.66
